<a href="https://colab.research.google.com/github/raffieeey/MasterResearchAutoML/blob/master/tpot/03A_TPOT_Regres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy scipy scikit-learn pandas joblib
!pip install deap update_checker tqdm stopit
!pip install xgboost
!pip install dask[delayed] dask-ml
!pip install scikit-mdr skrebate
!pip install tpot
!pip install pmlb
!pip install pytictoc

     |████████████████████████████████| 153kB 1.8MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-cp36-none-any.whl size=11955 sha256=ce350f0dcdcc8d10379a505865c11243d505896dee7d5dbb943262dcabbaadf3
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built stopit
     |████████████████████████████████| 122kB 1.7MB/s 
  Created wheel for scikit-mdr: filename=scikit_MDR-0.4.4-cp36-none-any.whl size=14639 sha256=2d34526bca82b987144277e48c5c94801222966989e210d2af68e1b45f3b9a90
  Stored in directory: /root/.cache/pip/wheels/ee/50/91/9d48f7807ff574cdbf8f82fc65b7075b23acc9106d3cced38f
  Created wheel for skrebate: filename=skrebate-0.6-cp36-none-any.whl size=29329 sha256=2cc58a5258cb831c86c12f2cf291960051ff1debff73c313cccfcafb37785ff4
  Stored in directory: /root/.cache/pip/wheels/f5/99/36/c827bcfa6852c6d068895b2723c57cea84f93642270c6dc05c
Successfully built scikit-mdr skrebate
     |████████████████████████████████| 8

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
import math
import numpy as np

class MetricsCalc:

    def __init__(self, truth, predictions):
        self.truth = pd.DataFrame(truth)
        self.predictions = predictions
        self._type = 'regression'

    def mae(self):
        return float(mean_absolute_error(self.truth, self.predictions))

    def mse(self):
        return float(mean_squared_error(self.truth, self.predictions))

    def msle(self):
      try:
        return float(mean_squared_log_error(self.truth, self.predictions))
      except:
        return np.nan
    
    def rmse(self):
        return math.sqrt(self.mse())

    def rmsle(self):
        return math.sqrt(self.msle())

    def r2(self):
        return float(r2_score(self.truth, self.predictions))

class CheckNegativeValue:
  def __init__(self, checknegative):
    self._is_negative = "neg" if np.min(checknegative) < 1 else "pos"
  
  def MLMetrics(self):
    if self._is_negative == "neg" :
      return ['neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']
    elif self._is_negative == "pos" :
      return ['neg_mean_squared_log_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']

In [7]:
from pmlb import classification_dataset_names, regression_dataset_names,fetch_data
from tpot import TPOTClassifier, TPOTRegressor
from sklearn.model_selection import train_test_split
from pytictoc import TicToc
import pandas as pd

df_result = pd.DataFrame(columns=["dataset_name", "metrics_recorded", "MAE", "MSE","MSLE", "RMSE", "RMSLE", "r2"])

preds = {}
cm_preds = {}
top_algo = {}
t = TicToc()


sel_clss_dtst = ['adult','agaricus-lepiota', 'churn', 'nursery', 'satimage','texture']
sel_rgrs_dtst = ['294_satellite_image','218_house_8L', '227_cpu_small', '503_wind', '344_mv','215_2dplanes']

ix = 0

for rgrs_dtst in sel_rgrs_dtst:
  for metrics_recorded  in ['neg_mean_squared_log_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'r2']:
        print("_"*80)
        print(f"*** Dataset Name: {rgrs_dtst} ***")
        df_result.loc[ix,"dataset_name"] = rgrs_dtst 
        df_result.loc[ix,"metrics_recorded"] = metrics_recorded 
        X, y = fetch_data(rgrs_dtst , return_X_y=True)
        c=CheckNegativeValue(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=69)
        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)
        y_train = pd.DataFrame(y_train).values.ravel()
        y_test = pd.DataFrame(y_test).values.ravel()
        list_metrics = c.MLMetrics()
        tpot = TPOTRegressor(random_state=69, max_time_mins=2, n_jobs=2,  scoring=metrics_recorded if metrics_recorded in list_metrics else None )
        t.tic()
        tpot.fit(X_train,y_train)
        t.toc()
        top_algo[rgrs_dtst] = [v[0] for i, v in tpot.pareto_front_fitted_pipelines_.items()][0]
        print("*** Top algorithm: ***", [v[0] for i, v in tpot.pareto_front_fitted_pipelines_.items()][0])
        preds[rgrs_dtst] = tpot.predict(X_test)
        p = MetricsCalc(y_test,preds[rgrs_dtst])
        df_result.loc[ix,'MAE'] = p.mae()
        df_result.loc[ix,'MSE'] = p.mse()
        df_result.loc[ix,'MSLE'] = p.msle()
        df_result.loc[ix,'RMSE'] = p.rmse()
        df_result.loc[ix,'RMSLE'] = p.rmsle()
        df_result.loc[ix,'r2'] = p.r2()
        ix+=1

________________________________________________________________________________
*** Dataset Name: 294_satellite_image ***
Elapsed time is 142.872262 seconds.
*** Top algorithm: *** FeatureUnion(n_jobs=None,
             transformer_list=[('functiontransformer-1',
                                FunctionTransformer(accept_sparse=False,
                                                    check_inverse=True,
                                                    func=<function copy at 0x7f45db113ae8>,
                                                    inv_kw_args=None,
                                                    inverse_func=None,
                                                    kw_args=None,
                                                    pass_y='deprecated',
                                                    validate=None)),
                               ('functiontransformer-2',
                                FunctionTransformer(accept_sparse=False,
                   

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Elapsed time is 133.475962 seconds.
*** Top algorithm: *** FeatureUnion(n_jobs=None,
             transformer_list=[('functiontransformer-1',
                                FunctionTransformer(accept_sparse=False,
                                                    check_inverse=True,
                                                    func=<function copy at 0x7f45db113ae8>,
                                                    inv_kw_args=None,
                                                    inverse_func=None,
                                                    kw_args=None,
                                                    pass_y='deprecated',
                                                    validate=None)),
                               ('functiontransformer-2',
                                FunctionTransformer(accept_sparse=False,
                                                    check_inverse=True,
                                                    func=<function cop

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Elapsed time is 133.972681 seconds.
*** Top algorithm: *** FeatureUnion(n_jobs=None,
             transformer_list=[('functiontransformer-1',
                                FunctionTransformer(accept_sparse=False,
                                                    check_inverse=True,
                                                    func=<function copy at 0x7f45db113ae8>,
                                                    inv_kw_args=None,
                                                    inverse_func=None,
                                                    kw_args=None,
                                                    pass_y='deprecated',
                                                    validate=None)),
                               ('functiontransformer-2',
                                FunctionTransformer(accept_sparse=False,
                                                    check_inverse=True,
                                                    func=<function cop

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Elapsed time is 134.481553 seconds.
*** Top algorithm: *** FeatureUnion(n_jobs=None,
             transformer_list=[('functiontransformer-1',
                                FunctionTransformer(accept_sparse=False,
                                                    check_inverse=True,
                                                    func=<function copy at 0x7f45db113ae8>,
                                                    inv_kw_args=None,
                                                    inverse_func=None,
                                                    kw_args=None,
                                                    pass_y='deprecated',
                                                    validate=None)),
                               ('functiontransformer-2',
                                FunctionTransformer(accept_sparse=False,
                                                    check_inverse=True,
                                                    func=<function cop

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


Elapsed time is 183.580372 seconds.
*** Top algorithm: *** RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=0.6500000000000001, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=12, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
________________________________________________________________________________
*** Dataset Name: 344_mv ***
Elapsed time is 182.946101 seconds.
*** Top algorithm: *** RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=0.6500000000000001, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=12, min_samples_split=2,
        

In [8]:
for i, val in enumerate(preds):
  print(preds[val])

[6.9283724  1.1998761  1.0335883  6.0901127  4.656174   2.892177
 0.9858165  0.9609588  3.4776404  2.549876   7.0309844  1.1175833
 4.4769163  1.9717728  2.0087786  5.01729    2.0087786  4.7678504
 0.5228519  3.0700116  2.9985223  5.8856297  0.9321513  3.1560333
 1.0128384  3.191826   5.614149   1.0243452  6.8454843  2.7167494
 3.1873655  1.0212733  1.9987376  4.5858464  5.685031   3.2904465
 2.7748199  1.206224   5.5045786  0.98142797 4.9303775  4.6962194
 0.94496185 5.8300242  0.7964618  0.80370325 2.738137   6.8569016
 1.9500144  3.0203469  6.396368   0.87559247 3.8113742  2.6012435
 3.324294   3.820649   3.2390432  1.7600737  4.543061   3.62501
 3.1065357  3.0480804  1.1030779  7.008935   1.0158391  7.0406795
 2.0049448  3.1970956  2.5826664  3.0115988  1.300867   2.0044832
 4.5191956  6.483885   6.8469806  1.0727451  2.5405693  5.0162086
 6.463084   7.0381026  3.0368297  7.389126   1.1486447  3.2838159
 6.6105547  6.9667864  4.82401    7.0615945  3.2732916  4.240098
 4.682295   3.

In [0]:
for i, val in enumerate(preds):
  pd.DataFrame(preds[val]).to_csv(f"{val}.csv")

In [10]:
df_result

,dataset_name,metrics_recorded,MAE,MSE,MSLE,RMSE,RMSLE,r2
0,294_satellite_image,neg_mean_squared_log_error,0.384834,0.512207,0.0215785,0.715686,0.146896,0.894131
1,294_satellite_image,neg_mean_absolute_error,0.386488,0.682943,0.0258829,0.826403,0.160882,0.858841
2,294_satellite_image,neg_mean_squared_error,0.447627,0.622583,0.02808,0.789039,0.167571,0.871317
3,294_satellite_image,r2,0.447627,0.622583,0.02808,0.789039,0.167571,0.871317
4,218_house_8L,neg_mean_squared_log_error,15857.4,8.88308e+08,0.404513,29804.5,0.636013,0.656786
5,218_house_8L,neg_mean_absolute_error,15857.4,8.88308e+08,0.404513,29804.5,0.636013,0.656786
6,218_house_8L,neg_mean_squared_error,15857.4,8.88308e+08,0.404513,29804.5,0.636013,0.656786
7,218_house_8L,r2,15857.4,8.88308e+08,0.404513,29804.5,0.636013,0.656786
8,227_cpu_small,neg_mean_squared_log_error,1.91968,7.33548,0.00145349,2.70841,0.0381247,0.976877
9,227_cpu_small,neg_mean_absolute_error,1.91968,7.33548,0.00145349,2.70841,0.0381247,0.976877


In [12]:
top_algo

{'215_2dplanes': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              importance_type='gain', learning_rate=1.0, max_delta_step=0,
              max_depth=2, min_child_weight=3, missing=None, n_estimators=100,
              n_jobs=1, nthread=1, objective='reg:squarederror', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8500000000000001, verbosity=1),
 '218_house_8L': RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                       max_features=0.5, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=19, min_samples_split=6,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_s